In [11]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)
seq_data = [['word', '단어'],['wood', '나무'],
            ['game', '놀이'],['girl', '소녀'],
            ['kiss', '키스'],['love', '사랑']]

# 입력단어와 출력단어를 한 글자씩 떼어낸 뒤 배열로 만든 후에 원-핫 인코딩 형식으로까지 만들어주는 유틸리티 함수를 만듬
# 1. 인코더 셀의 입력값을 위해 입력 단어를 한 글자씩 떼어 배열로 만듬.
# input=[num_dic[n] for n in seq[0]]
# 2. 디코더 셀의 입력값을 위해 출력 단어의 글자들을 배열로 만들고, 시작을 나타내는 심볼 'S'를 맨 앞에 붙임
# output = [num_dic[n] for n in ('S' +seq[1])]
# 3. 학습을 위해 비교할 디코더 셀의 출력값을 만들고, 출력의 끝을 알려주는 심볼 'E'를 마지막에 붙임
# target = [num_dic[n] for n in (seq[1]+ 'E')]
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for seq in seq_data:
        input = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])]
        target = [num_dic[n] for n in (seq[1] + 'E')]
        
        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    
    return input_batch, output_batch, target_batch

# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

n_class = n_input = dic_len

# 입력 값들은 원-핫 인코딩을 사용하고, 디코더의 출력값은 인덱스 숫자를 그대로 사용하기 때문에 입력값의 랭크(차원)가 하나 더 높음
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# 인코더 셀
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)
    
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    # 디코더를 만들때 초기 상태값이 아닌 인코더의 최종상태 값을 넣어줘야함
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state =enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch,
                                                    dec_input:output_batch,
                                                    targets: target_batch})
    print('Epoch:', '%04d' % (epoch +1), 'cost = ', '{:6f}'.format(loss))
    
print('최적화 완료!')

# 예측시에는 한글단어를 알지 못하기 때문에 PPPP로 채움
def translate(word):
    # 입력으로 word를 받으면 seq_data = ['word', 'PPPP']
    # input_batch는 ['w','o','r','d'], output_batch는 ['P','P','P','P']글자들의 인덱스를 원-핫 인코딩한 값, 
    # target_batch는 ['P','P','P','P'] 의 각 글자의 인덱스인 [2,2,2,2]
    seq_data = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    
    # 결과값이 [batch size, time steps, input size] 형태로 나오기 때문
    # 그리고 이 결과값이 [[[0 0 0.9 0.1, ... ][0 0.1 0.3 0.7, ...]]] 이런식으로 나오면  
    prediction = tf.argmax(model, 2)
    
    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})
    
    decoded = [char_arr[i] for i in result[0]]
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))

Epoch: 0001 cost =  3.753540
Epoch: 0002 cost =  2.705342
Epoch: 0003 cost =  1.511761
Epoch: 0004 cost =  1.084670
Epoch: 0005 cost =  0.539876
Epoch: 0006 cost =  0.653998
Epoch: 0007 cost =  0.372987
Epoch: 0008 cost =  0.217848
Epoch: 0009 cost =  0.216434
Epoch: 0010 cost =  0.166296
Epoch: 0011 cost =  0.269880
Epoch: 0012 cost =  0.166456
Epoch: 0013 cost =  0.053159
Epoch: 0014 cost =  0.056256
Epoch: 0015 cost =  0.046245
Epoch: 0016 cost =  0.042308
Epoch: 0017 cost =  0.014568
Epoch: 0018 cost =  0.076316
Epoch: 0019 cost =  0.007782
Epoch: 0020 cost =  0.034022
Epoch: 0021 cost =  0.015914
Epoch: 0022 cost =  0.013800
Epoch: 0023 cost =  0.007595
Epoch: 0024 cost =  0.007070
Epoch: 0025 cost =  0.010316
Epoch: 0026 cost =  0.012951
Epoch: 0027 cost =  0.005546
Epoch: 0028 cost =  0.002549
Epoch: 0029 cost =  0.004946
Epoch: 0030 cost =  0.003606
Epoch: 0031 cost =  0.009846
Epoch: 0032 cost =  0.004036
Epoch: 0033 cost =  0.004536
Epoch: 0034 cost =  0.001685
Epoch: 0035 co